# Prefix-Tuning 实战

## Step1 导入相关包

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

## Step2 加载数据集

In [ ]:
ds = Dataset.load_from_disk("../data/alpaca_data_zh/")
ds

In [ ]:
ds[:3]

## Step3 数据集预处理

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-1b4-zh")
tokenizer

In [ ]:
def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    response = tokenizer(example["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [ ]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

In [ ]:
tokenizer.decode(tokenized_ds[1]["input_ids"])

In [ ]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

## Step4 创建模型

In [ ]:
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh")

## Prefix-tuning

### PEFT Step1 配置文件

In [ ]:
from peft import PrefixTuningConfig, get_peft_model, TaskType

config = PrefixTuningConfig(task_type=TaskType.CAUSAL_LM,   ## 指定为因果语言建模任务
                            num_virtual_tokens=10,      # 虚拟token的数量 = 前缀长度
                            prefix_projection=True)     # 启用两层MLP投影：用两个线性层加上激活函数来变换前缀向量
'''
MLP其实就是线性变化，形状符合y=wx+b，而两层MLP就是把y1当作x2，带入y2方程里面，具体详见bilibili-多层感知机 + 代码实现 - 动手学深度学习v2
为什么需要两层？    数学上，这是通用近似定理的基础：一个至少包含一层隐藏层的神经网络，只要隐藏层有足够的神经元，就可以以任意精度逼近任何连续函数。
# 如果有两层（有激活函数）：
h = tanh(W1 * E + b1)
P = W2 * h + b2

维度是向量长度
# 二维向量（长度为2048）：
[0.1, 0.2, 0.3, ..., 0.2047, 0.2048]  # 2048个数值
# 这就像：
- 3维空间：需要x,y,z三个坐标
- 2048维空间：需要2048个坐标值

10个虚拟token是input的前缀，用于训练
而2048维度意为x或者y这个数组里面有2048个数（即关注这个事物2048个特征）,x经过第一层隐藏层的计算过程如下：
            # 输入：一个虚拟token的表示，假设是512维向量（如果projection_dim=512）
            x = [x1, x2, x3, ..., x512]  # 512个数值

            # 权重矩阵W1：形状[2048, 512]
            # 这表示有2048行，每行有512个权重
            W1 = [[w11, w12, ..., w1_512],   # 第1行：512个权重
                  [w21, w22, ..., w2_512],   # 第2行：512个权重
                  ...
                  [w2048_1, w2048_2, ..., w2048_512]]  # 第2048行：512个权重

            # 偏置b1：2048维向量
            b1 = [b1, b2, ..., b2048]

            # 计算y1 = W1 * x + b1
            # 结果y1是一个2048维向量：
            y1 = [y1_1, y1_2, ..., y1_2048]

            # 其中每个元素的计算：
            y1_1 = w11*x1 + w12*x2 + ... + w1_512*x512 + b1
            y1_2 = w21*x1 + w22*x2 + ... + w2_512*x512 + b2
            ...
            y1_2048 = w2048_1*x1 + w2048_2*x2 + ... + w2048_512*x512 + b2048

Prefix-Tuning在模型的每一层前面添加一些可学习的虚拟token（前缀），这些前缀向量会引导模型生成特定任务的输出。训练时只更新这些前缀向量，而冻结原始模型的所有参数。
'''
config

### PEFT Step2 创建模型

In [ ]:
model = get_peft_model(model, config)       ## 使用get_peft_model将基础模型转换为Prefix-Tuning模型

In [ ]:
model.prompt_encoder

In [ ]:
model.print_trainable_parameters()

## Step5 配置训练参数

In [ ]:
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=1
)

## Step6 创建训练器

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

## Step7 模型训练

In [ ]:
trainer.train()

## Step8 模型推理

In [ ]:
model = model.cuda()
ipt = tokenizer("Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(model.device)
tokenizer.decode(model.generate(**ipt, max_length=128, do_sample=True)[0], skip_special_tokens=True)